# Backtesting Futures

### Importing Packages

In [ ]:
from ib_insync import *
util.startLoop()
import pandas as pd
import numpy as np
from math import floor
from datetime import timedelta,datetime
import matplotlib.pyplot as plt
from Backtesting_Class import Backtesting_Strategy

In [ ]:
datetime.now()

### Definning variable

In [ ]:
instrument = 'AAPL'
hora_ini = '08:30:00'
hora_fin = '14:59:00'
target = 1.30
tempo = 1
num_bars = 1
tempo_h = 1/12
num_bars_h = int((num_bars*tempo)/tempo_h)

account = 20000
risk = 0.001
leverage = 1
commission = 0.47
tick_size = 0.01
slippage = 10

total_0 = []

In [ ]:
buy = False
sell = False
exit_sell = False
exit_buy= False
initial = False
exit_type_sell = ''
exit_type_buy = ''
profit_buy = 0
profit_sell = 0
max_high = 0
min_low = 0

In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 2, length = 100, fill = '#'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

### Download Data

In [ ]:
historical = pd.read_csv(str(instrument)+'_5secs.csv',parse_dates=['date'],index_col='date')

### Setting the initial and final date to get days of evaluation

In [ ]:
initial_date = '2018-06-06'
final_date = '2019-06-09'

delta = (pd.to_datetime(final_date) - pd.to_datetime(initial_date)).days + 1
print('Number of Evaluation days: ', delta)

In [ ]:
dates = [str((pd.to_datetime(initial_date) + timedelta(days=x)).strftime("%Y-%m-%d")) for x in range(delta)]

In [ ]:
total_0 = pd.DataFrame(total_0)

In [ ]:
iteration = 0

In [ ]:
#dates = ['2019/08/13', '2019/08/14','2019/08/15','2019/08/16','2019/08/20','2019/08/21','2019/08/23']
#delta = len(dates)

In [ ]:
for date in dates:
    #Getting weekday
    weekday = pd.to_datetime(date).weekday()

    #Getting the historical piece of data to evaluate
    hist = historical.loc[date+' '+hora_ini:date+' '+hora_fin,:].copy(deep=True)
    hour_range = pd.to_datetime(date+' '+hora_ini) + timedelta(minutes = num_bars*tempo)

    if not hist.empty and weekday < 5:
        bars_range = hist.loc[date+' '+hora_ini:,:].iloc[:num_bars_h]
        maximum = round(bars_range['high'].max(),2)
        minimum = round(bars_range['low'].min(),2)
        #target = 1.07
        lots = floor((account*risk)/((maximum-minimum)*leverage))
        
        if lots > 0:
            for i in range(hist.shape[0]):
                if hist.index[i] > hour_range:
                    current_high = hist.high[i]
                    current_low = hist.low[i]

                    if (current_high > maximum) and not buy:
                        buy = True
                        entry_price_buy = maximum + (slippage*tick_size)
                    if (current_low < minimum) and not sell:
                        sell = True
                        entry_price_sell = minimum - (slippage*tick_size)
                    
                    if not initial:
                        max_high = current_high
                        min_low = current_low
                        initial = True
                    
                    if buy and current_high > max_high and exit_type_buy != 'stop':
                        max_high = current_high
                    if sell and current_low < min_low and exit_type_sell != 'stop':
                        min_low = current_low

                    if buy and not exit_buy:
                        if current_low < minimum:
                            exit_buy = True
                            exit_price_buy = minimum - slippage*tick_size
                            exit_type_buy = 'stop'
                        elif current_high > maximum+target:
                            exit_buy = True
                            exit_price_buy = maximum+target
                            exit_type_buy = 'target'
                        #elif str(hist.index[i]) >= date+' '+hora_fin:
                        if i == hist.shape[0]-1:
                            exit_buy = True
                            exit_price_buy = hist.close[i] - slippage*tick_size
                            exit_type_buy = 'time'
                    if sell and not exit_sell:
                        if current_high > maximum:
                            exit_sell = True
                            exit_price_sell = maximum + slippage*tick_size
                            exit_type_sell = 'stop'
                        elif current_low < minimum-target:
                            exit_sell = True
                            exit_price_sell = minimum-target
                            exit_type_sell = 'target'
                        #elif str(hist.index[i]) >= date+' '+hora_fin:
                        if i == hist.shape[0]-1:
                            exit_sell = True
                            exit_price_sell = hist.close[i] + slippage*tick_size
                            exit_type_sell = 'time'
        if not buy:
            profit_buy_max = 0
            profit_buy = 0
        else:
            profit_buy = exit_price_buy - entry_price_buy
            profit_buy_max = max_high - entry_price_buy
        if not sell:
            profit_sell_max = 0
            profit_sell = 0
        else:
            profit_sell = entry_price_sell - exit_price_sell
            profit_sell_max = entry_price_sell - min_low

        #Getting results
        results = [date, profit_buy, profit_sell, profit_buy_max, profit_sell_max, lots]
        results = pd.DataFrame(results).T.set_index(0)

        #Appending results
        total_0 = pd.concat([total_0,results])
        
        #Restarting Variables
        maximum = minimum = 0
        lots = 0
        buy = sell = False
        exit_buy = exit_sell = False
        initial = False
        current_high = current_low = 0
        max_high = min_low = 0
        entry_price_buy = entry_price_sell = 0
        exit_price_buy = exit_price_sell = 0
        profit_buy = profit_sell = 0
        profit_buy_max = profit_sell_max = 0
        
    iteration+=1
    printProgressBar(iteration, len(dates) , prefix = 'Progress:', suffix = 'Complete', length = 50)

In [ ]:
#date = dates[4]
#date = dates[2]
date = '2019-08-23'
#Getting weekday
weekday = pd.to_datetime(date).weekday()

#Getting the historical piece of data to evaluate
hist = historical.loc[date+' '+hora_ini:date+' '+hora_fin,:].copy(deep=True)
hour_range = pd.to_datetime(date+' '+hora_ini) + timedelta(minutes = num_bars*tempo)

if not hist.empty and weekday < 5:
    bars_range = hist.loc[date+' '+hora_ini:,:].iloc[:num_bars_h]
    maximum = round(bars_range['high'].max(),2)
    minimum = round(bars_range['low'].min(),2)
    target = 5.30
    lots = floor((account*risk)/((maximum-minimum)*leverage))

    if lots > 0:
        for i in range(hist.shape[0]):
            if hist.index[i] > hour_range:
                current_high = hist.high[i]
                current_low = hist.low[i]

                if (current_high > maximum) and not buy:
                    buy = True
                    entry_price_buy = maximum + (slippage*tick_size)
                if (current_low < minimum) and not sell:
                    sell = True
                    entry_price_sell = minimum - (slippage*tick_size)

                if not initial:
                    max_high = current_high
                    min_low = current_low
                    initial = True
                
                if buy and current_high > max_high and exit_type_buy != 'stop':
                    max_high = current_high
                if sell and current_low < min_low and exit_type_sell != 'stop':
                    min_low = current_low

                if buy and not exit_buy:
                    if current_low < minimum:
                        exit_buy = True
                        exit_price_buy = minimum - slippage*tick_size
                        exit_type_buy = 'stop'
                    elif current_high > maximum+target:
                        exit_buy = True
                        exit_price_buy = maximum+target
                        exit_type_buy = 'target'
                    #elif str(hist.index[i]) >= date+' '+hora_fin:
                    if i == hist.shape[0]-1:
                        exit_buy = True
                        exit_price_buy = hist.close[i] - slippage*tick_size
                        exit_type_buy = 'time'
                if sell and not exit_sell:
                    if current_high > maximum:
                        exit_sell = True
                        exit_price_sell = maximum + slippage*tick_size
                        exit_type_sell = 'stop'
                    elif current_low < minimum-target:
                        exit_sell = True
                        exit_price_sell = minimum-target
                        exit_type_sell = 'target'
                    #elif str(hist.index[i]) >= date+' '+hora_fin:
                    if i == hist.shape[0]-1:
                        exit_sell = True
                        exit_price_sell = hist.close[i] + slippage*tick_size
                        exit_type_sell = 'time'
    if not buy:
        profit_buy_max = 0
        profit_buy = 0
    else:
        profit_buy = exit_price_buy - entry_price_buy
        profit_buy_max = max_high - entry_price_buy
    if not sell:
        profit_sell_max = 0
        profit_sell = 0
    else:
        profit_sell = entry_price_sell - exit_price_sell
        profit_sell_max = entry_price_sell - min_low

    #Getting results
    results = [date, profit_buy, profit_sell, profit_buy_max, profit_sell_max, lots]
    results = pd.DataFrame(results).T.set_index(0)

    #Appending results
    total_0 = pd.concat([total_0,results])

    #Restarting Variables
    '''maximum = minimum = 0
    lots = 0
    buy = sell = False
    exit_buy = exit_sell = False
    current_high = current_low = 0
    max_high = min_low = 0
    entry_price_buy = entry_price_sell = 0
    exit_price_buy = exit_price_sell = 0
    profit_buy = profit_sell = 0
    profit_buy_max = profit_sell_max = 0

iteration+=1
printProgressBar(iteration, len(dates) , prefix = 'Progress:', suffix = 'Complete', length = 50)'''

In [ ]:
#total_0

In [ ]:
date = '2019-08-23'
date

In [ ]:
'''hist = historical.loc[date+' '+hora_ini:date+' '+hora_fin,:].copy(deep=True)
bars_range = hist.loc[date+' '+hora_ini:,:].iloc[:num_bars_h]
maximum = round(bars_range['high'].max(),2)
minimum = round(bars_range['low'].min(),2)'''

In [ ]:
target

In [ ]:
'''hist.high.plot(figsize = (16,10),color = 'green')#.resample('5Min').max().plot(figsize = (16,10),color = 'green')
hist.low.plot(color = 'red')#.resample('5Min').min().plot(color = 'red')
plt.axhline(y=maximum, color='blue', linestyle='-')
plt.axhline(y=minimum, color='orange', linestyle='-')
plt.axhline(y=maximum+target, color='green', linestyle='-')
plt.axhline(y=minimum-target, color='green', linestyle='-')'''

In [ ]:
total_0

In [ ]:
total_0.index.names = ['date']
total_0.columns = ['final profit buy', 'final profit sell',
                 'max profit buy', 'max profit sell', 'lots']

In [ ]:
total = total_0

In [ ]:
#total = pd.read_excel('total_0.xlsx').set_index('date')

### Profit by day

In [ ]:
total['profit usd'] = (total['final profit buy'] + total['final profit sell']) * total['lots'] * leverage

### Acumulated Profit

In [ ]:
total['accumulated profit'] = total['profit usd'].cumsum() + account

In [ ]:
total['max profit'] = total['accumulated profit'].cummax()

### Instantiating Backtesting class

In [ ]:
back = Backtesting_Strategy(total)

### Backtesting Results

### 1. Total Profit (USD)

In [ ]:
total_profit_usd = back.final_profit_usd()
print('total profit (USD): ', total_profit_usd)

#### profit in ticks (mean)

In [ ]:
profit_buy_mean = round(((total['max profit buy']).loc[(total['max profit buy']) != 0]).mean(),2)
print('profit buy mean(ticks): ',profit_buy_mean)

In [ ]:
profit_sell_mean = round(((total['max profit sell']).loc[(total['max profit sell']) != 0]).mean(),2)
print('profit sell mean(ticks): ',profit_sell_mean)

In [ ]:
profit_mean = (profit_buy_mean + profit_sell_mean)/2
print('profit mean (ticks): ', profit_mean)

### 2. Total commissions (USD)

In [ ]:
total_commissions = back.total_commissions()
print('total commissions (USD): ', total_commissions)

### 3. Net profit

In [ ]:
net_profit = total_profit_usd - total_commissions
print('net profit (USD): ', net_profit)

### 4. Gross Profit and Gross Loss (USD)

In [ ]:
gross_profit, gross_loss = back.gross_profit_and_loss()
print('gross profit(USD): ', gross_profit, ' gross loss (USD): ',gross_loss)

### 5. Profit Factor

In [ ]:
profit_factor = round(abs(gross_profit/gross_loss),2)
print('profit factor: ', profit_factor)

### 6. Maximal Drawdown

In [ ]:
max_drawdown, max_draw_date = back.max_drawdown()
print('maximal drawdown(usd): ', max_drawdown, ' date: ', max_draw_date)

### 7. Relative Drawdown

In [ ]:
relative_drawdown = max_drawdown/account
print('Relative drawdown: ', round(relative_drawdown*100,2),'%')

### 8. Absolute Drawdown

In [ ]:
absolute_drawdown = back.absolute_drawdown(account)
print('Absolute Drawdown(usd):', absolute_drawdown)

### 9. Maximal Drawdown Period and date

In [ ]:
max_dd_days, dates_max_dd = back.max_drawdown_date(account)
print('Max. number of days in DD: ', max_dd_days, ', between days: ', dates_max_dd)

### 10. Trades Info

#### 10.1. Total transactions

In [ ]:
total_trades, total_positive, total_negative, percent_total = back.transactions_info('total')
print('total trades: ', total_trades, ' positive trades: ',total_positive, ' negative trades: ', total_negative,
      '\nTotal Percent profitable: ', round(percent_total*100,2),'%')

#### 10.2. Short transactions

In [ ]:
short_trades, positive_shorts, negative_shorts, percent_shorts = back.transactions_info('shorts')
print('total sell trades: ', short_trades, ' positive shorts: ',positive_shorts, ' negative shorts: ',negative_shorts,
      '\nPercent profitable shorts: ', round(percent_shorts*100,2),'%')

#### 10.3. Long transactions

In [ ]:
long_trades, positive_longs, negative_longs, percent_longs = back.transactions_info('longs')
print('total buy trades: ', long_trades, ' positive longs: ',positive_longs,  ' negative longs: ',negative_longs,
      '\nPercent profitable longs: ', round(percent_longs*100,2),'%')

### 11. Expected Payoff

In [ ]:
expected_payoff = round((total_profit_usd - total_commissions) / total_trades,2)
print('Expected Payoff: ', expected_payoff)

### 12. Greater Transactions

#### 12.1. Profitable

In [ ]:
greater_prof = back.greater_transactions('profitable')
print('Greater Profitable transaction: ' , greater_prof)

#### 12.1. Non Profitable

In [ ]:
greater_non_prof = back.greater_transactions('non profitable')
print('Greater non Profitable transaction: ' , greater_non_prof)

### 13. Average Transactions

#### 13.1. Profitable

In [ ]:
average_prof = back.average_transaction('profitable')
print('Average profitable transaction: ', average_prof)

#### 13.2. Non Profitable

In [ ]:
average_non_prof = back.average_transaction('non profitable')
print('Average non profitable transaction: ', average_non_prof)

### Final Metrics

In [ ]:
final_metrics = {}
final_metrics = {'Initial Account (usd)': account, 'Total profit (usd)': total_profit_usd, 'Total Commissions (usd)': total_commissions, 
                 'Net Profit (usd)': net_profit, 'Gross Profit (usd)': gross_profit, 'Gross Loss (usd)': gross_loss, 
                 'Profit Factor': profit_factor, 'Maximal Drawdown (usd)': max_drawdown, 'Maximal Drawdown Date': max_draw_date,
                 'Relative Drawdown (%)': round(relative_drawdown*100,2), 'Absolute Drawdown (usd)': absolute_drawdown,  
                 'Maximal period in Drawdown (days)': max_dd_days, 'Dates of max. DD period': str(dates_max_dd), 
                 'Total Transactions': total_trades, 'Winning Trades': total_positive, 'Winning Trades (%)': round(percent_total*100,2),
                 'Losing Trades': total_negative, 'Losing Trades (%)': round(100-(percent_total*100),2),
                 'Short Positions': short_trades, 'Winning Shorts': positive_shorts, 'Winning Shorts (%)': round(percent_shorts*100,2),
                 'Losing Shorts': negative_shorts, 'Losing Shorts(%)': round(100-(percent_shorts*100),2),
                 'Long Positions': long_trades, 'Winning Longs': positive_longs, 'Winning Longs (%)': round(percent_longs*100,2),
                 'Losing Longs:': negative_longs, 'Losing Longs(%)': round(100-(percent_longs*100),2),
                 'Expected Payoff (usd)': expected_payoff,
                 'Greater Profitable Transaction (usd)': greater_prof, 'Greater non Profitable Transaction (usd)': greater_non_prof,
                 'Profitable Transaction Average (usd)': average_prof, 'Non Profitable Transaction Average (usd)': average_non_prof}

In [ ]:
final_metrics = pd.DataFrame(final_metrics, index = [0]).T
final_metrics.index.names = ['Metric']
final_metrics.columns = ['Values']
final_metrics

### Saving Results

In [ ]:
final_metrics.to_excel('Backtesting_stocks_recent/'+instrument+'_'+str(tempo)+'Min_'+str(num_bars)+'_'+str(target)+'.xlsx')

In [ ]:
total.to_excel('Backtesting_stocks_recent/total_'+instrument+'_'+str(tempo)+'Min.xlsx')

In [ ]:
total['accumulated profit'].plot(figsize=(18,10),label='accumulated',color='black',lw=2)
total['max profit'].plot(color='red',label='max',ls='-',alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Profit')
plt.title('Results '+instrument+' '+str(tempo)+' Min')
plt.legend()
plt.savefig('Backtesting_stocks_recent/'+instrument+'_'+str(tempo)+'Min.png')

In [ ]:
datetime.now()